In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
# Create sample dataset1

data1 = pd.DataFrame({
    'Name': ['Amar', 'Amar', 'Ajay', 'Ajay', 'Kapil', 'Suraj', 'Nitin', 'Deepak', 'Deepak', 'Deepak', 'Ram'],
    'student_idNum': ['123', '124', '222', '203', '333', '421', '511', '685', '679', '612', '712'],
    'end_date': ['2021-01-10','2021-02-10','2020-01-10','2021-10-10','2021-11-10','2021-03-10',
                 '2024-01-10','2021-01-10','2021-05-10','2022-01-10','2023-01-10'],
    'suspended': [1,1,0,0,0,1,1,1,1,0,0],
    'deleted_at': ['2021-01-11','2021-02-12',np.NaN,np.NaN,np.NaN,'2021-03-10',
                 np.NaN,'2021-01-10','2021-05-10',np.NaN,np.NaN]
})
new_dtypes = {"Name": str, "student_idNum": int, "end_date": np.datetime64, "suspended": int, "deleted_at": np.datetime64}
data1 = data1.astype(new_dtypes)
data1["Active"] = (data1['end_date'].le(pd.to_datetime(date.today())) | data1["suspended"] == 1 | pd.isnull(data1['deleted_at'])).astype(int)
data1

,Name,student_idNum,end_date,suspended,deleted_at,Active
0,Amar,123,2021-01-10,1,2021-01-11,1
1,Amar,124,2021-02-10,1,2021-02-12,1
2,Ajay,222,2020-01-10,0,NaT,1
3,Ajay,203,2021-10-10,0,NaT,0
4,Kapil,333,2021-11-10,0,NaT,0
5,Suraj,421,2021-03-10,1,2021-03-10,1
6,Nitin,511,2024-01-10,1,NaT,1
7,Deepak,685,2021-01-10,1,2021-01-10,1
8,Deepak,679,2021-05-10,1,2021-05-10,1
9,Deepak,612,2022-01-10,0,NaT,0


In [3]:
# Create sample dataset2

data2 = pd.DataFrame({
    'Name': ['Ajay', 'Amar', 'Deepak', 'Kapil', 'Nitin', 'Ram', 'Suraj'], 
    'student_idText': ['A222^A222^A223', 'A123^A129^A124', 'D612^D612^D679^D689', 
                       'K333^3Kapil', 'N511^5Nitin', 'R712^7Ram', 'S421^4Suraj'] 
})
data2

,Name,student_idText
0,Ajay,A222^A222^A223
1,Amar,A123^A129^A124
2,Deepak,D612^D612^D679^D689
3,Kapil,K333^3Kapil
4,Nitin,N511^5Nitin
5,Ram,R712^7Ram
6,Suraj,S421^4Suraj


In [4]:
# Create a mapping table - to match the 'id numbers' against 'id text' values

mapping = pd.DataFrame({
    'student_idMap': ['123', '124', '222', '223', '333', '421', '511', '612', '679', '685', '712'], 
'student_idText': ['A123', 'A124', 'A222', 'A223', 'K333', 'S421', 'N511', 'D612', 'D679', 'D685', 'R712'] 
})
new_dtypes = {"student_idMap": int, "student_idText": str}
mapping = mapping.astype(new_dtypes)
mapping

,student_idMap,student_idText
0,123,A123
1,124,A124
2,222,A222
3,223,A223
4,333,K333
5,421,S421
6,511,N511
7,612,D612
8,679,D679
9,685,D685


In [5]:
# Merge dataset1 and mapping data

data11 = pd.merge(data1,
                    mapping,
                    how='left',  
                    left_on='student_idNum', 
                    right_on='student_idMap',
                    left_index=False,
                    right_index=False,
                   validate='one_to_many').drop('student_idMap', axis=1)
data11

,Name,student_idNum,end_date,suspended,deleted_at,Active,student_idText
0,Amar,123,2021-01-10,1,2021-01-11,1,A123
1,Amar,124,2021-02-10,1,2021-02-12,1,A124
2,Ajay,222,2020-01-10,0,NaT,1,A222
3,Ajay,203,2021-10-10,0,NaT,0,NaN
4,Kapil,333,2021-11-10,0,NaT,0,K333
5,Suraj,421,2021-03-10,1,2021-03-10,1,S421
6,Nitin,511,2024-01-10,1,NaT,1,N511
7,Deepak,685,2021-01-10,1,2021-01-10,1,D685
8,Deepak,679,2021-05-10,1,2021-05-10,1,D679
9,Deepak,612,2022-01-10,0,NaT,0,D612


In [6]:
# Split combined studentidText into rows and remove duplicate rows

data22 = pd.DataFrame(data2.student_idText.str.split('^').tolist(), index = data2.Name).stack().reset_index([0, 'Name']).drop_duplicates()
data22.columns = ['Name','student_idText']
data22

,Name,student_idText
0,Ajay,A222
2,Ajay,A223
3,Amar,A123
4,Amar,A129
5,Amar,A124
6,Deepak,D612
8,Deepak,D679
9,Deepak,D689
10,Kapil,K333
11,Kapil,3Kapil


## Check for common records

In [7]:
# Check for common records

pd.merge(data11, data22, left_on=['Name','student_idText'], right_on=['Name','student_idText'], how='inner')

,Name,student_idNum,end_date,suspended,deleted_at,Active,student_idText
0,Amar,123,2021-01-10,1,2021-01-11,1,A123
1,Amar,124,2021-02-10,1,2021-02-12,1,A124
2,Ajay,222,2020-01-10,0,NaT,1,A222
3,Kapil,333,2021-11-10,0,NaT,0,K333
4,Suraj,421,2021-03-10,1,2021-03-10,1,S421
5,Nitin,511,2024-01-10,1,NaT,1,N511
6,Deepak,679,2021-05-10,1,2021-05-10,1,D679
7,Deepak,612,2022-01-10,0,NaT,0,D612
8,Ram,712,2023-01-10,0,NaT,0,R712


## Check for the missing records in first table

In [8]:
data111 = data11.copy()
data111

,Name,student_idNum,end_date,suspended,deleted_at,Active,student_idText
0,Amar,123,2021-01-10,1,2021-01-11,1,A123
1,Amar,124,2021-02-10,1,2021-02-12,1,A124
2,Ajay,222,2020-01-10,0,NaT,1,A222
3,Ajay,203,2021-10-10,0,NaT,0,NaN
4,Kapil,333,2021-11-10,0,NaT,0,K333
5,Suraj,421,2021-03-10,1,2021-03-10,1,S421
6,Nitin,511,2024-01-10,1,NaT,1,N511
7,Deepak,685,2021-01-10,1,2021-01-10,1,D685
8,Deepak,679,2021-05-10,1,2021-05-10,1,D679
9,Deepak,612,2022-01-10,0,NaT,0,D612


In [9]:
data111['key'] = 'x'
data111

,Name,student_idNum,end_date,suspended,deleted_at,Active,student_idText,key
0,Amar,123,2021-01-10,1,2021-01-11,1,A123,x
1,Amar,124,2021-02-10,1,2021-02-12,1,A124,x
2,Ajay,222,2020-01-10,0,NaT,1,A222,x
3,Ajay,203,2021-10-10,0,NaT,0,NaN,x
4,Kapil,333,2021-11-10,0,NaT,0,K333,x
5,Suraj,421,2021-03-10,1,2021-03-10,1,S421,x
6,Nitin,511,2024-01-10,1,NaT,1,N511,x
7,Deepak,685,2021-01-10,1,2021-01-10,1,D685,x
8,Deepak,679,2021-05-10,1,2021-05-10,1,D679,x
9,Deepak,612,2022-01-10,0,NaT,0,D612,x


In [10]:
temp_df1 = pd.merge(data111, data22, on = ['Name','student_idText'], how='right')
temp_df1

,Name,student_idNum,end_date,suspended,deleted_at,Active,student_idText,key
0,Ajay,222.0,2020-01-10,0.0,NaT,1.0,A222,x
1,Ajay,NaN,NaT,NaN,NaT,NaN,A223,NaN
2,Amar,123.0,2021-01-10,1.0,2021-01-11,1.0,A123,x
3,Amar,NaN,NaT,NaN,NaT,NaN,A129,NaN
4,Amar,124.0,2021-02-10,1.0,2021-02-12,1.0,A124,x
5,Deepak,612.0,2022-01-10,0.0,NaT,0.0,D612,x
6,Deepak,679.0,2021-05-10,1.0,2021-05-10,1.0,D679,x
7,Deepak,NaN,NaT,NaN,NaT,NaN,D689,NaN
8,Kapil,333.0,2021-11-10,0.0,NaT,0.0,K333,x
9,Kapil,NaN,NaT,NaN,NaT,NaN,3Kapil,NaN


In [11]:
temp_df1[temp_df1['key'].isnull()].drop(['student_idNum','end_date','suspended','deleted_at','Active','key'], axis=1)

,Name,student_idText
1,Ajay,A223
3,Amar,A129
7,Deepak,D689
9,Kapil,3Kapil
11,Nitin,5Nitin
13,Ram,7Ram
15,Suraj,4Suraj


## Check for the missing records in second table

In [12]:
data222 = data22.copy()
data222

,Name,student_idText
0,Ajay,A222
2,Ajay,A223
3,Amar,A123
4,Amar,A129
5,Amar,A124
6,Deepak,D612
8,Deepak,D679
9,Deepak,D689
10,Kapil,K333
11,Kapil,3Kapil


In [13]:
data222['key'] = 'x'
data222

,Name,student_idText,key
0,Ajay,A222,x
2,Ajay,A223,x
3,Amar,A123,x
4,Amar,A129,x
5,Amar,A124,x
6,Deepak,D612,x
8,Deepak,D679,x
9,Deepak,D689,x
10,Kapil,K333,x
11,Kapil,3Kapil,x


In [14]:
temp_df2 = pd.merge(data222, data11, on = ['Name','student_idText'], how='right')
temp_df2

,Name,student_idText,key,student_idNum,end_date,suspended,deleted_at,Active
0,Amar,A123,x,123,2021-01-10,1,2021-01-11,1
1,Amar,A124,x,124,2021-02-10,1,2021-02-12,1
2,Ajay,A222,x,222,2020-01-10,0,NaT,1
3,Ajay,NaN,NaN,203,2021-10-10,0,NaT,0
4,Kapil,K333,x,333,2021-11-10,0,NaT,0
5,Suraj,S421,x,421,2021-03-10,1,2021-03-10,1
6,Nitin,N511,x,511,2024-01-10,1,NaT,1
7,Deepak,D685,NaN,685,2021-01-10,1,2021-01-10,1
8,Deepak,D679,x,679,2021-05-10,1,2021-05-10,1
9,Deepak,D612,x,612,2022-01-10,0,NaT,0


In [15]:
temp_df2[temp_df2['key'].isnull()].drop('key', axis=1)

,Name,student_idText,student_idNum,end_date,suspended,deleted_at,Active
3,Ajay,NaN,203,2021-10-10,0,NaT,0
7,Deepak,D685,685,2021-01-10,1,2021-01-10,1
